<a href="https://colab.research.google.com/github/joampc/TFM2023/blob/main/notebooks/Modelo_1_TokenizadoSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=01d06e777c140eaf59f3eca17a4bc137b6d72150e83be1bfe39972612627cfb0
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling ch

In [2]:
import nltk
import multiprocessing
from functools import partial
from nltk.tokenize import word_tokenize
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import random
from googletrans import Translator
from scipy.special import softmax



In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_raw = pd.read_csv('/content/drive/MyDrive/TFM/Dataset Definitivo/RAW_recipes.csv')

print(f'Número de filas: {df_raw.shape[0]}, Número de columnas:{df_raw.shape[1]}')

Número de filas: 231637, Número de columnas:12


In [ ]:
df_raw.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6


In [ ]:
pd.set_option('display.max_colwidth', None)

print(df_raw.loc[:1, 'ingredients'])

0    ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
1           ['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']
Name: ingredients, dtype: object


#**Variante 1**

In [14]:
df_rawp = df_raw[:2000]

In [6]:
# Descarga los recursos necesarios para la tokenización (solo necesitas hacer esto una vez)
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Preprocesamiento de datos

##Tokenizacion

In [15]:

#Definimos una función para tokenizar una lista de texto (ingredientes o pasos)
def tokenize_list(text_list):
    # Removemos los corchetes y comillas del texto y luego dividimos por la coma
    return [ingredient.lower().strip().strip("'") for ingredient in text_list.strip("[]").split(",")]


df_rawp['ingredient_tokens'] = df_rawp['ingredients'].apply(tokenize_list)
df_rawp['steps_tokens'] = df_rawp['steps'].apply(tokenize_list)


# Visualización del DataFrame con las columnas tokenizadas
#print(df_raw[['name', 'ingredient_tokens', 'steps_tokens']])


<ipython-input-15-db03de39da58>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rawp['ingredient_tokens'] = df_rawp['ingredients'].apply(tokenize_list)
<ipython-input-15-db03de39da58>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rawp['steps_tokens'] = df_rawp['steps'].apply(tokenize_list)


In [26]:
pd.set_option('display.max_colwidth', None)
print(df_rawp.loc[:1, 'ingredients'])
print(df_rawp.loc[:1, 'ingredient_tokens'])

#df_rawp.head(2)

0    ['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']
1           ['prepared pizza crust', 'sausage patty', 'eggs', 'milk', 'salt and pepper', 'cheese']
Name: ingredients, dtype: object
0    [winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt]
1         [prepared pizza crust, sausage patty, eggs, milk, salt and pepper, cheese]
Name: ingredient_tokens, dtype: object


In [8]:
print(df_rawp.loc[:1, 'steps'])
print(df_rawp.loc[:1, 'steps_tokens'])


0    ['make a choice and proceed with recipe', 'dep...
1    ['preheat oven to 425 degrees f', 'press dough...
Name: steps, dtype: object
0    [make a choice and proceed with recipe, depend...
1    [preheat oven to 425 degrees f, press dough in...
Name: steps_tokens, dtype: object


##Creación de vocabulario

Crea un vocabulario a partir de todas las palabras únicas en los ingredientes y pasos. Asigna un índice numérico único a cada palabra para que el modelo pueda trabajar con valores numéricos.

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Creación del vocabulario
all_texts = df_rawp['ingredient_tokens'].tolist() + df_rawp['steps_tokens'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)


##Creación de secuencias numéricas:



Convierte las secuencias de ingredientes y pasos tokenizados en secuencias numéricas utilizando el vocabulario creado. Reemplaza cada palabra en las secuencias con su índice numérico correspondiente.

In [17]:
# Creación de secuencias numéricas
ingredient_sequences = tokenizer.texts_to_sequences(df_rawp['ingredient_tokens'])
steps_sequences = tokenizer.texts_to_sequences(df_rawp['steps_tokens'])


##Padding

Asegúrate de que todas las secuencias de ingredientes y pasos tengan la misma longitud. Esto es necesario para poder alimentar los datos al modelo en lotes.
Agrega tokens de relleno al final de las secuencias más cortas para que todas tengan la misma longitud máxima.

In [18]:
# Padding
max_sequence_length = max(max(len(seq) for seq in ingredient_sequences), max(len(seq) for seq in steps_sequences))
ingredient_sequences = pad_sequences(ingredient_sequences, maxlen=max_sequence_length, padding='post')
steps_sequences = pad_sequences(steps_sequences, maxlen=max_sequence_length, padding='post')

##División de datos:

Separa los datos en conjuntos de entrenamiento, validación y prueba. Es común usar aproximadamente un 80% de los datos para entrenamiento, un 10% para validación y un 10% para pruebas.

In [19]:
# División de datos en conjuntos de entrenamiento, validación y prueba

train_size = int(0.8 * len(df_raw))
val_size = int(0.1 * len(df_raw))
train_ingredients = ingredient_sequences[:train_size]
train_steps = steps_sequences[:train_size]
val_ingredients = ingredient_sequences[train_size:train_size+val_size]
val_steps = steps_sequences[train_size:train_size+val_size]
test_ingredients = ingredient_sequences[train_size+val_size:]
test_steps = steps_sequences[train_size+val_size:]


# También puedes obtener el vocabulario creado por el Tokenizer y los índices de palabras utilizando:
word_index = tokenizer.word_index



# Modelo

##Construcción del modelo:

In [20]:
#Comprobaciones que hicieron falta hasta q logre que funcionara

# Verificar las formas de los datos de entrenamiento y etiquetas
print("Forma de train_ingredients:", train_ingredients.shape)
print("Forma de train_steps:", train_steps.shape)

# Verificar los tipos de datos
print("Tipo de train_ingredients:", train_ingredients.dtype)
print("Tipo de train_steps:", train_steps.dtype)

# Verificar la longitud máxima de las secuencias de ingredientes y pasos
print("Longitud máxima de secuencias de ingredientes:", train_ingredients.shape[1])
print("Longitud máxima de secuencias de pasos:", train_steps.shape[1])

# Verificar el vocabulario y las etiquetas
vocab_size = len(word_index) + 1
print("Tamaño del vocabulario:", vocab_size)

# Imprime algunas etiquetas para verificar que sean enteros y estén en el rango correcto
print("Etiquetas de entrenamiento:", train_steps[:10])


#La forma de train_ingredients es (1000, 85), lo que significa que tienes 1000 ejemplos de recetas,
#cada una representada por una secuencia de ingredientes con una longitud máxima de 85 palabras.
#La forma de train_steps es también (1000, 85), lo que indica que tienes 1000 ejemplos de recetas,
# cada una representada por una secuencia de pasos con una longitud máxima de 85 palabras.
#Ambos arrays tienen el tipo de dato int32, que es el tipo de dato adecuado para datos de enteros.
#La longitud máxima de secuencias para ingredientes y pasos es 85, lo que significa que todas las recetas
# han sido acotadas a esta longitud máxima. Si algunas recetas tenían más de 85 palabras,
#se truncaron para que todas las secuencias tengan la misma longitud.

#El tamaño del vocabulario es 13794, lo que significa que hay 13794 palabras únicas en tus datos.

Forma de train_ingredients: (2000, 149)
Forma de train_steps: (2000, 149)
Tipo de train_ingredients: int32
Tipo de train_steps: int32
Longitud máxima de secuencias de ingredientes: 149
Longitud máxima de secuencias de pasos: 149
Tamaño del vocabulario: 25539
Etiquetas de entrenamiento: [[3904 3905 3906 ...    0    0    0]
 [1306 3923 3924 ...    0    0    0]
 [3931 3932 1307 ...    0    0    0]
 ...
 [3960 3961 3962 ...    0    0    0]
 [3975 3976 3977 ...    0    0    0]
 [  78 3985 3986 ...    0    0    0]]


Este código se encarga de crear y entrenar un modelo codificador-decodificador para generar recetas a partir de ingredientes. Explicaré cada paso en detalle:

1. **Creación del vocabulario:**
   - Se obtiene la lista completa de tokens de ingredientes y pasos de todas las recetas.
   - Se crea un `Tokenizer`, que se utiliza para transformar el texto en secuencias de números enteros y construir el vocabulario basado en la frecuencia de las palabras.
   - El método `fit_on_texts` del `Tokenizer` se utiliza para ajustar el vocabulario en función de todas las secuencias de texto proporcionadas.

2. **Verificación de tokens especiales:**
   - Se verifica si los tokens especiales '<start>' y '<end>' están presentes en el vocabulario. Estos tokens se utilizan para marcar el inicio y el final de las secuencias de texto generadas. Si no están presentes, se agregan manualmente al vocabulario.

3. **Obtención del tamaño del vocabulario:**
   - Se obtiene el tamaño del vocabulario que se utilizará en la capa de incrustación del modelo.

4. **Preprocesamiento de los datos:**
   - Se asegura de que la longitud máxima de ingredientes y pasos sea la misma para que puedan combinarse adecuadamente durante el entrenamiento.
   - Rellena las secuencias de ingredientes y pasos con ceros para que tengan la misma longitud máxima.

5. **Definición del modelo codificador-decodificador:**
   - Se define la arquitectura del modelo codificador-decodificador utilizando la API funcional de Keras.
   - El modelo utiliza dos capas LSTM (codificador y decodificador) para procesar secuencias de texto y generar recetas.
   - Se utilizan capas de incrustación para aprender representaciones vectoriales de palabras.
   - La última capa es una capa densa con una activación softmax para generar las probabilidades de la siguiente palabra en la secuencia.

6. **Compilación del modelo:**
   - Se compila el modelo utilizando el optimizador 'adam' y la función de pérdida 'sparse_categorical_crossentropy'. El modelo se evaluará en función de la precisión (accuracy) durante el entrenamiento.

7. **Ajuste de la forma de train_steps_padded:**
   - Se ajusta la forma de `train_steps_padded` para que pueda utilizarse adecuadamente en el entrenamiento. Se agrega una columna de ceros al final de cada secuencia.

8. **Entrenamiento del modelo:**
   - Se entrena el modelo utilizando los ingredientes (`train_ingredients_padded`) como entrada y los pasos de las recetas (`train_steps_padded[:, :-1]`) como salida esperada. El objetivo es predecir la siguiente palabra en la secuencia de pasos.
   - El modelo se entrena durante 50 épocas con un tamaño de lote de 32.

En resumen, este código crea y entrena un modelo de generación de recetas utilizando una arquitectura codificador-decodificador con capas LSTM y una capa densa de salida con activación softmax. El modelo se entrena para predecir la siguiente palabra en la secuencia de pasos de recetas a partir de los ingredientes proporcionados.

===============================================================================

En el contexto del modelo codificador-decodificador que se está utilizando para generar recetas, las capas de incrustación se refieren a las capas de Embedding en Keras. La incrustación (embedding en inglés) es un proceso mediante el cual se transforman las palabras en vectores numéricos densos y continuos, de modo que las palabras con significados similares se representen por vectores cercanos en un espacio vectorial.

En el código proporcionado, se utilizan dos capas de incrustación:

encoder_embedding: Esta capa de incrustación se aplica a la secuencia de ingredientes (encoder_input) para aprender representaciones vectoriales de las palabras en los ingredientes. Esto se hace para que el modelo pueda entender mejor la relación entre los ingredientes y los pasos de la receta.
decoder_embedding: Esta capa de incrustación se aplica a la secuencia de pasos (decoder_input) para aprender representaciones vectoriales de las palabras en los pasos de la receta. Esto es esencial para que el modelo pueda generar pasos coherentes y significativos en la receta.
Las capas de incrustación aprenden los vectores de palabras durante el proceso de entrenamiento del modelo codificador-decodificador. Cada palabra en el vocabulario se asigna a un vector numérico que se actualiza iterativamente a medida que el modelo se ajusta a los datos de entrenamiento. Al final del entrenamiento, el modelo ha aprendido representaciones vectoriales que reflejan las relaciones semánticas y sintácticas entre las palabras del vocabulario. Estas representaciones incrustadas permiten al modelo generar recetas coherentes y relevantes basadas en los ingredientes proporcionados.





Regenerate

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense
import numpy as np

# Creación del vocabulario
all_texts = df_rawp['ingredient_tokens'].tolist() + df_rawp['steps_tokens'].tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

# Verificar si '<start>' y '<end>' están presentes en el vocabulario, si no, agrégales manualmente
if '<start>' not in tokenizer.word_index:
    tokenizer.word_index['<start>'] = len(tokenizer.word_index) + 1
    tokenizer.index_word[len(tokenizer.word_index)] = '<start>'

if '<end>' not in tokenizer.word_index:
    tokenizer.word_index['<end>'] = len(tokenizer.word_index) + 1
    tokenizer.index_word[len(tokenizer.word_index)] = '<end>'

# Obtener el tamaño del vocabulario del tokenizer
vocab_size = len(tokenizer.word_index) + 1

# Asegurar que la longitud máxima de ingredientes y pasos sea la misma
max_sequence_length = 85

# Rellenar secuencias con ceros para que todas tengan la misma longitud
train_ingredients_padded = pad_sequences(train_ingredients, maxlen=max_sequence_length, padding='post')
train_steps_padded = pad_sequences(train_steps, maxlen=max_sequence_length, padding='post')

# Definir la arquitectura del modelo codificador-decodificador
embedding_dim = 100
rnn_units = 128

encoder_input = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_input)
encoder_lstm = LSTM(rnn_units)(encoder_embedding)

decoder_input = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_input)
decoder_lstm = LSTM(rnn_units, return_sequences=True)(decoder_embedding)
decoder_output = Dense(vocab_size, activation='softmax')(decoder_lstm)

# Crear el modelo codificador-decodificador
model = Model([encoder_input, decoder_input], decoder_output)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ajustar la forma de train_steps_padded
train_steps_padded = np.pad(train_steps_padded, [(0, 0), (0, 1)], mode='constant', constant_values=0)

# Entrenar el modelo con más épocas (por ejemplo, 50 épocas)
model.fit([train_ingredients_padded, train_steps_padded[:, :-1]], train_steps_padded[:, 1:], epochs=50, batch_size=32)


Epoch 1/50
63/63 [==============================] - 201s 3s/step - loss: 4.1633 - accuracy: 0.9827
Epoch 2/50
63/63 [==============================] - 192s 3s/step - loss: 0.1234 - accuracy: 0.9986
Epoch 3/50
63/63 [==============================] - 199s 3s/step - loss: 0.0769 - accuracy: 0.9986
Epoch 4/50
63/63 [==============================] - 190s 3s/step - loss: 0.0516 - accuracy: 0.9986
Epoch 5/50
63/63 [==============================] - 188s 3s/step - loss: 0.0309 - accuracy: 0.9986
Epoch 6/50
63/63 [==============================] - 197s 3s/step - loss: 0.0244 - accuracy: 0.9987
Epoch 7/50
63/63 [==============================] - 190s 3s/step - loss: 0.0206 - accuracy: 0.9986
Epoch 8/50
63/63 [==============================] - 191s 3s/step - loss: 0.0177 - accuracy: 0.9987
Epoch 9/50
63/63 [==============================] - 197s 3s/step - loss: 0.0165 - accuracy: 0.9987
Epoch 10/50
63/63 [==============================] - 188s 3s/step - loss: 0.0186 - accuracy: 0.9987
Epoch 11/

## Uso del modelo para generar una receta


In [22]:
# para poder imprimir en párrafos
!pip install textwrap3


In [23]:
import numpy as np
import textwrap


def softmax(x, temperature=1.0):
    e_x = np.exp((x - np.max(x)) / temperature)
    return e_x / e_x.sum(axis=0)

def generate_recipe_from_ingredients(model, tokenizer, ingredients_list, max_recipe_length=85, end_token='<end>', temperature=0.8):
    start_token = '<start>'
    step_sequence = [tokenizer.word_index[start_token]]

    # Convertir los ingredientes a secuencias numéricas utilizando el tokenizer
    ingredients_sequence = tokenizer.texts_to_sequences([ingredients_list])
    initial_ingredients_list = pad_sequences(ingredients_sequence, maxlen=max_recipe_length, padding='post')

    # Generar los pasos de la receta
    for _ in range(max_recipe_length - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_recipe_length, padding='post', truncating='post')
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence) - 1]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs, temperature=temperature)
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Convertir los índices de las palabras a texto
    recipe_text = ' '.join(tokenizer.index_word[idx] for idx in step_sequence[1:])  # Excluir el token de inicio
    return recipe_text

# Utilizando la función para generar una receta a partir de una lista de ingredientes
ingredients_list = ['fish', 'onion', 'garlic', 'potatoes']
recipe = generate_recipe_from_ingredients(model, tokenizer, ingredients_list)



#print("Receta generada:", recipe)

# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)


1/1 [==============================] - 0s 156ms/step

Receta generada:
place on cookie sheet "add the corn / oat mixture to the remaining egg whites
after 7 days mix in the flour and salt - kneading is unnecessary you will end up
with a big multi-layered cake or sandwich of omelettes ! store in tupperware
until ready to cook "shape the loaves by turning the dough under it's self over
and over" frozen broccoli cuts slowly add the brown sugar and granulated white
sugar flatten with a glass or cookie stamp roma tomatoes place on a baking sheet
and let rise for 1 hour then bake for 10 minutes allowing it to flow down and
cover the sides mix the dressing ingredients together with a wire whisk this is
great for a crock pot "don't use store bought powder place 1 scoop chocolate ice
cream next and add generous scoop of marshmallow creme strawberry: omit peanut
butter and add 1 1 / 2 cups frozen strawberries at the initial blending salt and
tapioca in small bowl and then sprinkle over all the m

In [24]:
#para mostrar los ingredientes y recetas en español
translator = Translator()

def translate_to_spanish(text):
    translation = translator.translate(text, src='en', dest='es')
    return translation.text


Generación de la receta:

La función comienza inicializando una secuencia de pasos de receta step_sequence con el token especial <start>, que indica el inicio de una receta.
Convierte los tokens de ingredientes seleccionados a secuencias numéricas utilizando el tokenizer proporcionado.
Establece la longitud máxima de la receta (max_recipe_length) como el mínimo entre max_steps y la longitud de la secuencia de ingredientes seleccionados más uno (para tener en cuenta el token especial <start>).
Luego, rellena o trunca la secuencia de ingredientes seleccionados para que tenga la misma longitud que max_recipe_length.
El bucle for se encarga de generar los pasos de la receta. En cada iteración:
Se rellena o trunca la secuencia de pasos de receta (step_sequence_input) para que tenga la misma longitud que max_recipe_length.
Se obtienen las probabilidades de las siguientes palabras en la secuencia de pasos utilizando el modelo entrenado (model.predict).
Se utiliza una temperatura para suavizar la distribución de probabilidad y generar un índice de palabra aleatorio.
Se verifica que el índice generado esté dentro del rango válido del vocabulario.
Se agrega la siguiente palabra a la secuencia de pasos (step_sequence).
Si la siguiente palabra es el token especial <end>, se detiene la generación de la receta.
Finalmente, la secuencia de pasos de receta se convierte en texto utilizando el tokenizer.

**MAX LENGHT**
La idea principal detrás de max_recipe_length es controlar la longitud de la receta generada. Para ello, se considera la longitud de random_tokens_sequence, que representa los ingredientes seleccionados. La receta generada debe comenzar con un token especial <start> y luego continuar con los pasos generados. Por lo tanto, la longitud máxima de la receta se establece como el mínimo entre max_steps (el número máximo de pasos permitidos) y la longitud de random_tokens_sequence más uno (para tener en cuenta el token especial <start> al principio de la receta).


**TEMPERATURA**
La temperatura es un parámetro importante en el proceso de generación de texto utilizando modelos de lenguaje como el que estamos utilizando aquí. En general, se utiliza para controlar la aleatoriedad y diversidad de las predicciones generadas por el modelo.

Cuando se genera texto con un modelo de lenguaje, el modelo produce distribuciones de probabilidad para cada palabra en el vocabulario en función del contexto actual. La probabilidad de cada palabra indica la "confianza" del modelo en que esa palabra sea la siguiente en la secuencia de texto.

Una temperatura alta (por ejemplo, 1.0) hace que la distribución de probabilidad sea más uniforme, lo que significa que las predicciones son más aleatorias y diversas. En otras palabras, el modelo tiene más libertad para seleccionar palabras menos probables y generar resultados más creativos pero también menos precisos.

Por otro lado, una temperatura baja (por ejemplo, 0.1) hace que la distribución de probabilidad sea más puntiaguda, lo que significa que las predicciones son más deterministas y se inclinan hacia las palabras más probables. En este caso, el modelo tiende a ser más conservador y genera resultados más precisos pero menos sorprendentes.

En resumen, una temperatura alta da lugar a predicciones más aleatorias y diversas, mientras que una temperatura baja da lugar a predicciones más precisas y deterministas. La elección de la temperatura depende del resultado deseado: si se busca creatividad e innovación, una temperatura alta puede ser adecuada, mientras que si se busca coherencia y precisión, una temperatura baja puede ser preferible. Generalmente, valores alrededor de 0.5 a 1.0 se utilizan para mayor diversidad y valores alrededor de 0.1 a 0.5 para mayor precisión.

**En esta función generate_recipe_from_random_ingredients, se generan los pasos de la receta utilizando el modelo codificador-decodificador, pero los pasos se agregan uno por uno en cada iteración del bucle. Esto no garantiza que los 5 ingredientes aleatorios seleccionados se utilicen en la receta generada**

In [25]:
import random


translator = Translator()

def translate_to_spanish(text):
    translation = translator.translate(text, src='en', dest='es')
    return translation.text


def generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=10, end_token='<end>', temperature=0.8):
    # Obtener la lista completa de tokens de ingredientes
    all_ingredients = df['ingredient_tokens'].tolist()
    all_tokens = [token for ingredients in all_ingredients for token in ingredients]

    # Seleccionar 5 tokens aleatorios
    random_tokens = random.sample(all_tokens, num_ingredients)

    print("Ingredientes seleccionados:")
    for i, token in enumerate(random_tokens):
        english_ingredient = token
        spanish_ingredient = translate_to_spanish(token)
        print(f"Ingrediente {i+1}: {english_ingredient} / {spanish_ingredient}")




    start_token = '<start>'
    step_sequence = [tokenizer.word_index[start_token]]

    # Convertir los tokens seleccionados a secuencias numéricas utilizando el tokenizer
    random_tokens_sequence = tokenizer.texts_to_sequences([random_tokens])[0]

    # Encontrar la receta más larga entre las seleccionadas y usarla como longitud máxima
    max_recipe_length = min(max_steps, len(random_tokens_sequence) + 1)

    # Rellenar o truncar la secuencia de ingredientes para que tenga la misma longitud que max_recipe_length
    initial_ingredients_list = pad_sequences([random_tokens_sequence], maxlen=max_recipe_length - 1, padding='post', truncating='post')

    # Generar los pasos de la receta
    for _ in range(max_steps - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_recipe_length, padding='post', truncating='post')
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence_input[0]) - 1]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs, temperature=temperature)
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio
    recipe_text = ' '.join(tokenizer.index_word.get(idx, '') for idx in step_sequence[1:])

      # Convertir los índices de las palabras a texto, excluyendo el token de inicio, y traducir a español
    recipe_sp = ' '.join(translate_to_spanish(tokenizer.index_word.get(idx, '')) for idx in step_sequence[1:])

    return recipe_text, recipe_sp

# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe, recipe_sp = generate_recipe_from_random_ingredients(model, tokenizer, df_rawp, num_ingredients=5, max_steps=8, temperature=0.1)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')
lines_sp = wrapper.wrap(recipe_sp)
for line in lines_sp:
    print(line)



Ingredientes seleccionados:
Ingrediente 1: sugar / azúcar
Ingrediente 2: flour / harina
Ingrediente 3: water / agua
Ingrediente 4: ground coriander / cilantro
Ingrediente 5: hot sauce / salsa picante
1/1 [==============================] - 0s 25ms/step

Receta generada:
3 tablespoons brown sugar and 15 caramels until smooth mix the melted caramels
with the icing sugar and 3 tablespoons milk with an electric mixer until smooth
when edges begin to bubble and lift from skillet crisco shortening as soon as
the leavening ingredients are combined if it appears dry and stiff or if you
machine sounds as if it's straining to knead it buy the long box not the country
layer 5: celery


3 cucharadas de azúcar morena y 15 caramelos hasta que mezcle suave los
caramelos derretidos con el azúcar glas y 3 cucharadas de leche con una batidora
eléctrica hasta que estén suaves Cuando los bordes comienzan a burbujear y
levantar de la sartén acortamiento de crisco Tan pronto como se combinan los
ingredientes

In [26]:
import random
import textwrap
from googletrans import Translator

translator = Translator()

def translate_to_spanish(text):
    translation = translator.translate(text, src='en', dest='es')
    return translation.text

def generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=10, end_token='<end>', temperature=0.8):
    # Obtener la lista completa de tokens de ingredientes
    all_ingredients = df['ingredient_tokens'].tolist()
    all_tokens = [token for ingredients in all_ingredients for token in ingredients]

    # Seleccionar 5 tokens aleatorios
    random_tokens = random.sample(all_tokens, num_ingredients)

    print("Ingredientes seleccionados:")
    for i, token in enumerate(random_tokens):
        english_ingredient = token
        spanish_ingredient = translate_to_spanish(token)
        print(f"Ingrediente {i+1}: {english_ingredient} / {spanish_ingredient}")

    # Convertir los tokens seleccionados a secuencias numéricas utilizando el tokenizer
    random_tokens_sequence = tokenizer.texts_to_sequences([random_tokens])[0]

    # Encontrar la receta más larga entre las seleccionadas y usarla como longitud máxima
    max_recipe_length = min(max_steps, len(random_tokens_sequence) + 1)

    # Rellenar o truncar la secuencia de ingredientes para que tenga la misma longitud que max_recipe_length
    initial_ingredients_list = pad_sequences([random_tokens_sequence], maxlen=max_recipe_length - 1, padding='post', truncating='post')

    # Generar los pasos de la receta
    step_sequence = [tokenizer.word_index['<start>']] + list(random_tokens_sequence)
    for _ in range(max_steps - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_recipe_length, padding='post', truncating='post')
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence_input[0]) - 1]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs, temperature=temperature)
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio
    recipe_text = ' '.join(tokenizer.index_word.get(idx, '') for idx in step_sequence[1:])

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio, y traducir a español
    recipe_sp = ' '.join(translate_to_spanish(tokenizer.index_word.get(idx, '')) for idx in step_sequence[1:])

    return recipe_text, recipe_sp

# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe, recipe_sp = generate_recipe_from_random_ingredients(model, tokenizer, df_rawp, num_ingredients=5, max_steps=10, temperature=0.5)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')
lines_sp = wrapper.wrap(recipe_sp)
for line in lines_sp:
    print(line)


Ingredientes seleccionados:
Ingrediente 1: boiling water / agua hirviendo
Ingrediente 2: tomato paste / pasta de tomate
Ingrediente 3: parmesan cheese / queso parmesano
Ingrediente 4: sugar / azúcar
Ingrediente 5: clove / Clavo
1/1 [==============================] - 0s 40ms/step

Receta generada:
boiling water tomato paste parmesan cheese sugar clove to freeze: allow soup to
cool put the green peas on top of the spinach or mixed greens refrigerate for
about 10 to 15 minutes before cutting into 2 inch squares set aside 1 / 2 cup to
1 cup of the greens sausage mixture for the top "some folks like to serve this
hot instead of cold: bake at 350 degrees f for 30 minutes stir until
marshmallows are all melted in a large dutch oven with a tight fitting lid these
are so easy and goes great with salsa flip [carefully] when you see bubbles in
the middle of the pancake


agua hirviendo pasta de tomate queso parmesano azúcar Clavo Para congelar: deja
que la sopa se enfríe Pon los guisantes verdes 

In [27]:
import random

def translate_to_spanish(text):
    translation = translator.translate(text, src='en', dest='es')
    return translation.text


def generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=12, end_token='<end>', temperature=0.5):
    # Obtener la lista completa de tokens de ingredientes
    all_ingredients = df['ingredient_tokens'].tolist()
    all_tokens = [token for ingredients in all_ingredients for token in ingredients]

    # Seleccionar 5 tokens aleatorios
    random_tokens = random.sample(all_tokens, num_ingredients)

    print("Ingredientes seleccionados:")
    for i, token in enumerate(random_tokens):
        english_ingredient = token
        spanish_ingredient = translate_to_spanish(token)
        print(f"Ingrediente {i+1}: {english_ingredient} / {spanish_ingredient}")

    # Convertir los tokens seleccionados a secuencias numéricas utilizando el tokenizer
    random_tokens_sequence = tokenizer.texts_to_sequences([random_tokens])[0]

    # Encontrar la receta más larga entre las seleccionadas y usarla como longitud máxima
    max_recipe_length = min(max_steps, len(random_tokens_sequence) + 1)

    # Rellenar o truncar la secuencia de ingredientes para que tenga la misma longitud que max_recipe_length
    initial_ingredients_list = pad_sequences([random_tokens_sequence], maxlen=max_recipe_length - 1, padding='post', truncating='post')

    # Generar los pasos de la receta
    step_sequence = [tokenizer.word_index['<start>']] + list(random_tokens_sequence)
    for _ in range(max_steps - num_ingredients - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_recipe_length, padding='post', truncating='post')
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence_input[0]) - 1]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs, temperature=temperature)
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio
    recipe_text = ' '.join(tokenizer.index_word.get(idx, '') for idx in step_sequence[1:])

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio, y traducir a español
    recipe_sp = ' '.join(translate_to_spanish(tokenizer.index_word.get(idx, '')) for idx in step_sequence[1:])

    return recipe_text, recipe_sp


# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe, recipe_sp = generate_recipe_from_random_ingredients(model, tokenizer, df_rawp, num_ingredients=5, max_steps=12, temperature=0.5)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')
lines_sp = wrapper.wrap(recipe_sp)
for line in lines_sp:
    print(line)


Ingredientes seleccionados:
Ingrediente 1: frozen chopped spinach / espinacas picadas congeladas
Ingrediente 2: flour / harina
Ingrediente 3: cream of mushroom soup / crema de sopa de hongos
Ingrediente 4: lean ground beef / carne de res molida
Ingrediente 5: semi-sweet chocolate chips / chips de chocolate semidulce
1/1 [==============================] - 0s 35ms/step

Receta generada:
frozen chopped spinach flour cream of mushroom soup lean ground beef semi-sweet
chocolate chips saute onions until they are clear and soft add the pumpkin and
all the remaining ingredients except the last 2 and bring to a simmer knead for
6-8 minutes add flour alternating with 7-up currants half and slice your apple


espinacas picadas congeladas harina crema de sopa de hongos carne de res molida
chips de chocolate semidulce saltee las cebollas hasta que estén claras y suaves
Agregue la calabaza y todos los ingredientes restantes, excepto los últimos 2 y
llevar a fuego lento amasar durante 6-8 minutos Agr

In [50]:
def generate_recipe_from_random_ingredients(model, tokenizer, df, num_ingredients=5, max_steps=12, end_token='<end>', temperature=0.5):
    # Obtener la lista completa de tokens de ingredientes
    all_ingredients = df['ingredient_tokens'].tolist()
    all_tokens = [token for ingredients in all_ingredients for token in ingredients]

    # Seleccionar 5 tokens aleatorios
    random_tokens = random.sample(all_tokens, num_ingredients)

    print("Ingredientes seleccionados:")
    for i, token in enumerate(random_tokens):
        english_ingredient = token
        spanish_ingredient = translate_to_spanish(token)
        print(f"Ingrediente {i+1}: {english_ingredient} / {spanish_ingredient}")

    # Convertir los tokens seleccionados a secuencias numéricas utilizando el tokenizer
    random_tokens_sequence = tokenizer.texts_to_sequences([random_tokens])[0]

    # Rellenar o truncar la secuencia de ingredientes para que tenga la misma longitud que max_steps
    initial_ingredients_list = pad_sequences([random_tokens_sequence], maxlen=max_steps - 1, padding='post', truncating='post')

    # Generar los pasos de la receta
    step_sequence = [tokenizer.word_index['<start>']]
    for _ in range(max_steps - num_ingredients - 1):
        step_sequence_input = pad_sequences([step_sequence], maxlen=max_steps, padding='post', truncating='post')
        next_token_probs = model.predict([initial_ingredients_list, step_sequence_input])[0, len(step_sequence)]

        # Utilizar temperatura para suavizar la distribución de probabilidad
        next_token_probs = softmax(next_token_probs, temperature=temperature)
        # Normalizar las probabilidades para que sumen 1
        next_token_probs /= next_token_probs.sum()

        # Muestrear el siguiente token utilizando las probabilidades normalizadas
        next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        # Verificar que el índice generado esté dentro del rango válido del vocabulario
        while next_token_index == 0 or tokenizer.index_word.get(next_token_index) is None:
            next_token_index = np.random.choice(len(next_token_probs), p=next_token_probs)

        next_token = tokenizer.index_word[next_token_index]
        step_sequence.append(next_token_index)

        if next_token == end_token:
            break

    # Agregar los ingredientes seleccionados a los pasos de la receta generada
    for i, token_idx in enumerate(random_tokens_sequence):
        step_sequence.insert(i+1, token_idx)

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio
    recipe_text = ' '.join(tokenizer.index_word.get(idx, '') for idx in step_sequence[1:])

    # Convertir los índices de las palabras a texto, excluyendo el token de inicio, y traducir a español
    recipe_sp = ' '.join(translate_to_spanish(tokenizer.index_word.get(idx, '')) for idx in step_sequence[1:])

    return recipe_text, recipe_sp


# Utilizando la función para generar una receta con 5 ingredientes aleatorios
recipe, recipe_sp = generate_recipe_from_random_ingredients(model, tokenizer, df_rawp, num_ingredients=5, max_steps=12, temperature=0.5)
# Imprimir la receta con wordwrap
print("\nReceta generada:")
wrapper = textwrap.TextWrapper(width=80)
lines = wrapper.wrap(recipe)
for line in lines:
    print(line)
print('\n')
lines_sp = wrapper.wrap(recipe_sp)
for line in lines_sp:
    print(line)

Ingredientes seleccionados:
Ingrediente 1: banana / banana
Ingrediente 2: tomatoes / Tomates
Ingrediente 3: black beans / frijoles negros
Ingrediente 4: condensed french onion soup / sopa de cebolla francesa condensada
Ingrediente 5: salt / sal
1/1 [==============================] - 0s 30ms/step

Receta generada:
banana tomatoes black beans condensed french onion soup salt mix everything
except asparagus and set aside leave it in the mixer drain and return meat
mixture to skillet break up the brittle in small pieces and place into a food
processor stir in flour until well blended but not boiling


banana Tomates frijoles negros sopa de cebolla francesa condensada sal Mezclar
todo excepto los espárragos y reservar Déjalo en la batidora Drene y regrese la
mezcla de carne a la sartén Rompe el quebradiza en trozos pequeños y colóquelo
en un procesador de alimentos Agregue la harina hasta que esté bien mezclado
pero no hirviendo
